In [1]:
%load_ext autoreload
%autoreload 2

import s3fs
import zarr
import xarray as xr

import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import torch
import gpytorch
import cartopy.crs as ccrs

from herbie import Herbie

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

import net, model, penalty, optimize

/home/yf297/.conda/envs/myenv/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [2]:
Lambert_proj = ccrs.LambertConformal(central_longitude=262.5, central_latitude=38.5,
                                standard_parallels=[38.5,38.5],
                                globe=ccrs.Globe(semimajor_axis=6371229, 
                                                 semiminor_axis=6371229))

In [3]:
paths = []
n = 24

for i in range(0,n):
    paths.append(Herbie("2024-08-07", model="hrrr", fxx=i).download("500 mb"))
    

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Aug-07 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Aug-07 00:00 UTC F01 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Aug-07 00:00 UTC F02 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Aug-07 00:00 UTC F03 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Aug-07 00:00 UTC F04 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Aug-07 00:00 UTC F05 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Aug-07 00:00 UTC F06 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Aug-07 00:00 UTC F07 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Aug-07 00:00 UTC F08 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Aug-07 00:00 UTC F09 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Aug-07 00:00 UTC F10 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr 

In [4]:
def process(path, i):
    ds = xr.open_dataset(path, engine='cfgrib')

    ds.expand_dims(dim={"time": [i]})
    ds.attrs = {}
    return ds

In [14]:
data = xr.concat([process(paths[i], i) for i in range(n)], 
                 dim='time')

/home/yf297/.conda/envs/myenv/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/home/yf297/.conda/envs/myenv/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/home/yf297/.conda/envs/myenv/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attr

In [15]:
data["longitude"] = data["longitude"] - 360
Lambert_proj.transform_points(ccrs.Geodetic(), 
                              data.longitude.values,
                              data.latitude.values)

XY = Lambert_proj.transform_points(ccrs.Geodetic(), 
                              data.longitude.values,
                              data.latitude.values)[:,:,0:2]

data = data.assign_coords(x=("x",  XY[0,:,0]))
data = data.assign_coords(y=("y",  XY[:,0,1]));

In [16]:
(l,d) = Lambert_proj.transform_point(-88,20,ccrs.Geodetic())
(r,u) = Lambert_proj.transform_point(-40,23,ccrs.Geodetic())
data_sub0 = data.sel(x = slice(l,r), y = slice(d,u))

In [17]:
data_sub0 = data_sub0.isel(time = slice(0,6,1))


In [18]:
data_sub0

<xarray.Dataset> Size: 42MB
Dimensions:        (time: 6, y: 562, x: 552)
Coordinates:
  * time           (time) datetime64[ns] 48B 2024-08-07 ... 2024-08-07
    step           (time) timedelta64[ns] 48B 00:00:00 01:00:00 ... 05:00:00
    isobaricInhPa  float64 8B 500.0
    latitude       (y, x) float64 2MB 23.87 23.87 23.87 ... 35.36 35.35 35.34
    longitude      (y, x) float64 2MB -87.53 -87.5 -87.47 ... -67.31 -67.28
    valid_time     (time) datetime64[ns] 48B 2024-08-07 ... 2024-08-07T05:00:00
  * x              (x) float64 4kB 1.043e+06 1.046e+06 ... 2.693e+06 2.696e+06
  * y              (y) float64 4kB -1.587e+06 -1.584e+06 ... 9.269e+04 9.569e+04
Data variables:
    gh             (time, y, x) float32 7MB 5.932e+03 5.932e+03 ... 5.906e+03
    t              (time, y, x) float32 7MB 269.4 269.4 269.4 ... 266.3 266.3
    dpt            (time, y, x) float32 7MB 257.4 257.4 257.4 ... 242.8 242.8
    u              (time, y, x) float32 7MB 2.632 2.694 2.694 ... 1.233 1.171
    v              (time, y, x) float32 7MB -2.201 -2.201 -2.201 ... 10.35 10.35

In [19]:
fig, ax = plt.subplots()
mesh = ax.imshow(np.zeros_like(data_sub0.dpt.values[0,:,:]), cmap="coolwarm", origin = "lower")

def update(frame):
    mesh.autoscale()
    mesh.set_data(data_sub0.dpt.values[frame,:,:])
    return mesh,

ani = animation.FuncAnimation(
    fig,
    update,
    frames=data_sub0.dpt.values.shape[0],  
    interval=100,         
    blit=False
)

plt.close(fig)

HTML(ani.to_jshtml())

In [19]:
data_sub = data_sub0.isel(y=slice(None, None, 35), x=slice(None, None, 35))

k1 = data_sub.dpt.shape[1:][0]
k2 = data_sub.dpt.shape[1:][1]
m = k1*k2
n =  data_sub.dpt.shape[0]

In [20]:
data_sub

<xarray.Dataset> Size: 220kB
Dimensions:        (time: 6, y: 31, x: 52)
Coordinates:
  * time           (time) datetime64[ns] 48B 2024-08-07 ... 2024-08-07
    step           (time) timedelta64[ns] 48B 00:00:00 01:00:00 ... 05:00:00
    isobaricInhPa  float64 8B 500.0
    latitude       (y, x) float64 13kB 21.14 21.38 21.61 ... 48.46 48.13 47.78
    longitude      (y, x) float64 13kB -122.7 -121.8 -120.8 ... -62.8 -61.51
    valid_time     (time) datetime64[ns] 48B 2024-08-07 ... 2024-08-07T05:00:00
  * x              (x) float64 416B -2.698e+06 -2.593e+06 ... 2.657e+06
  * y              (y) float64 248B -1.587e+06 -1.482e+06 ... 1.563e+06
Data variables:
    gh             (time, y, x) float32 39kB 5.888e+03 5.89e+03 ... 5.63e+03
    t              (time, y, x) float32 39kB 268.7 268.6 268.2 ... 255.8 256.7
    dpt            (time, y, x) float32 39kB 254.1 252.2 252.9 ... 234.8 233.7
    u              (time, y, x) float32 39kB -5.056 -6.181 -7.868 ... 25.8 27.42
    v              (time, y, x) float32 39kB 0.5489 -0.1386 ... 9.286 11.54

In [21]:
fig, ax = plt.subplots()
mesh = ax.imshow(np.zeros_like(data_sub.dpt.values[0,:,:]), cmap="coolwarm", origin = "lower")

def update(frame):
    mesh.autoscale()
    mesh.set_data(data_sub.dpt.values[frame,:,:])
    return mesh,

ani = animation.FuncAnimation(
    fig,
    update,
    frames=data_sub.dpt.values.shape[0],  
    interval=100,         
    blit=False
)

plt.close(fig)

HTML(ani.to_jshtml())

In [22]:
Z = torch.tensor(data_sub.dpt.values, dtype=torch.float32).reshape(-1) - 273.15
Z = (Z - Z.mean())/7.5

Z_tensor = Z.reshape(n, k1, k2).flatten(1)

In [23]:
Z.min(), Z.max()

(tensor(-7.1729), tensor(2.5521))

In [ ]:
grid = torch.tensor(Lambert_proj.transform_points(ccrs.Geodetic(), 
                              data_sub.longitude.values,
                              data_sub.latitude.values)[:,:,0:2], dtype=torch.float32)/1e6

TXY = torch.cat((torch.linspace(0,1,n).repeat_interleave(m).unsqueeze(1), 
                   XY.reshape(-1,2).repeat(n,1)), dim = -1)


In [26]:
XY.shape

torch.Size([31, 52, 2])

In [113]:
flow = net.Flow(L = 9)
vel = net.Vel(L = 3)
gp_flow = model.GP_FLOW(TXY, Z_tensor, flow, vel)


In [114]:
distst = torch.cdist(gp_flow.T.unsqueeze(1), gp_flow.T.unsqueeze(1))  
distsx = torch.cdist(gp_flow.XY[:,0:1], gp_flow.XY[:,0:1])  
distsy = torch.cdist(gp_flow.XY[:,1:2], gp_flow.XY[:,1:2])  

In [115]:
dists = torch.cdist(gp_flow.XY, gp_flow.XY)  

gp_flow.gp.Kernel.base_kernel.initialize(lengthscale= torch.tensor([distst.max(),
                                                                    distsx.median()/4, 
                                                                    distsy.median()/4]))

gp_flow.gp.Kernel.initialize(outputscale=Z_tensor[0].var());
gp_flow.gp.Kernel.outputscale, gp_flow.gp.Kernel.base_kernel.lengthscale, gp_flow.likelihood.noise

(tensor(1.7815, grad_fn=<SoftplusBackward0>),
 tensor([[1.0000, 0.3937, 0.2363]], grad_fn=<SoftplusBackward0>),
 tensor([0.6932], grad_fn=<AddBackward0>))

In [116]:
import gc
gc.collect()

3877

In [117]:
torch.cuda.empty_cache()

In [118]:
with gpytorch.settings.fast_computations(log_prob=False):
    optimize.initial(gp_flow, num_epochs=75)

Epoch: 25 - Likelihood: 0.864
Epoch: 50 - Likelihood: 0.861
Epoch: 75 - Likelihood: 0.861


In [119]:
gp_flow.gp.Kernel.outputscale, gp_flow.gp.Kernel.base_kernel.lengthscale, gp_flow.likelihood.noise

(tensor(1.4487, grad_fn=<SoftplusBackward0>),
 tensor([[1.0000, 0.3938, 0.2717]], grad_fn=<SoftplusBackward0>),
 tensor([0.1388], grad_fn=<AddBackward0>))

In [120]:
#with gpytorch.settings.fast_computations(log_prob=False):
optimize.gp(gp_flow, num_epochs=120)

Epoch: 10 - Likelihood: 0.501
Epoch: 20 - Likelihood: 0.468
Epoch: 30 - Likelihood: 0.452
Epoch: 40 - Likelihood: 0.441
Epoch: 50 - Likelihood: 0.434
Epoch: 60 - Likelihood: 0.428
Epoch: 70 - Likelihood: 0.423
Epoch: 80 - Likelihood: 0.419
Epoch: 90 - Likelihood: 0.416
Epoch: 100 - Likelihood: 0.412
Epoch: 110 - Likelihood: 0.409
Epoch: 120 - Likelihood: 0.407


In [121]:
gp_flow.gp.Kernel.outputscale, gp_flow.gp.Kernel.base_kernel.lengthscale, gp_flow.likelihood.noise

(tensor(1.4487, grad_fn=<SoftplusBackward0>),
 tensor([[5.1799, 0.3938, 0.2717]], grad_fn=<SoftplusBackward0>),
 tensor([0.1388], grad_fn=<AddBackward0>))

In [122]:
#optimize.project(gp_flow)

In [126]:
vels = penalty.v_hat(gp_flow.TXY, gp_flow.flow, 0.3).reshape(n,
                                                        k1,
                                                        k2,
                                                        2).detach()

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.set_aspect('equal')

s = 2
Q_hat = ax.quiver(XY[::s,::s,0].detach(),
                  XY[::s,::s,1].detach(), 
                  np.ones_like(vels[0, ::s, ::s, 0]), 
                  np.ones_like(vels[0, ::s, ::s, 1]),
                  color="red", 
                  angles='xy', 
                  scale_units='xy', 
                  scale=1.5)

def update(frame):
    U_hat = vels[frame, ::s, ::s, 0] 
    V_hat = vels[frame, ::s, ::s, 1] 
    Q_hat.set_UVC(U_hat, V_hat)
    return Q_hat,


ani = animation.FuncAnimation(fig, update, frames=n, blit=False)
plt.close()

HTML(ani.to_jshtml())

In [100]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.set_aspect('equal')

s = 2
Q_hat = ax.quiver(XY[::s,::s,0].detach(),
                  XY[::s,::s,1].detach(), 
                  np.ones_like(vels[0, ::s, ::s, 0]), 
                  np.ones_like(vels[0, ::s, ::s, 1]),
                  color="red", 
                  angles='xy', 
                  scale_units='xy', 
                  scale=80)

def update(frame):
    U_hat = data_sub.u[frame, ::s, ::s]
    V_hat = data_sub.v[frame, ::s, ::s]
    Q_hat.set_UVC(U_hat, V_hat)
    return Q_hat,


ani = animation.FuncAnimation(fig, update, frames=n, blit=False)
plt.close()

HTML(ani.to_jshtml())

In [82]:
data_sub.u.shape

(4, 71, 42)

In [110]:
np.min(abs(data_sub.u))

<xarray.DataArray 'u' ()> Size: 4B
array(0.00691414, dtype=float32)
Coordinates:
    isobaricInhPa  float64 8B 500.0